In [ ]:
!pip install faker kafka-python

In [ ]:
use role accountadmin; 

create database if not exists charger;

create schema if not exists kafka_poc;

use schema charger.kafka_poc;

use warehouse compute_wh;

-- create or replace table LINEITEM cluster by (L_SHIPDATE) as select * from "SNOWFLAKE_SAMPLE_DATA"."TPCH_SF1000"."LINEITEM" limit 2000000;
-- create or replace table ORDERS as select * from "SNOWFLAKE_SAMPLE_DATA"."TPCH_SF1000"."ORDERS" limit 100000;

In [3]:
import pandas as pd
import random
from faker import Faker
from random import randrange
from datetime import datetime
from time import sleep
from json import dumps
from kafka import KafkaProducer

In [7]:
nr_of_customers = 3

fake = Faker("de_DE")

customers = dict()

topic_name = "sales-data"

producer = KafkaProducer(
    bootstrap_servers=["localhost:9092"],
    value_serializer=lambda x: dumps(x).encode("utf-8"),
)

for customers_id in range(nr_of_customers):
    # Create transaction date
    d1 = datetime.strptime(f"1/1/2021", "%m/%d/%Y")
    d2 = datetime.strptime(f"8/10/2021", "%m/%d/%Y")
    transaction_date = fake.date_between(d1, d2)

    # create customer's name
    name = fake.name()

    # Create gender
    gender = random.choice(["M", "F"])

    # Create email
    email = fake.ascii_email()

    # Create city
    city = fake.city()

    # create product ID in 8-digit barcode
    product_id = fake.ean(length=8)

    # create amount spent
    amount_spent = fake.pyfloat(
        right_digits=2, positive=True, min_value=1, max_value=100
    )

    customers = {
        "transaction_date": str(transaction_date),
        "name": name,
        "gender": gender,
        "city": city,
        "email": email,
        "product_id": product_id,
        "amount_spent": amount_spent,
    }
    print(customers)

    producer.send(topic_name, value=dumps(customers))
    sleep(5)

{'transaction_date': '2021-08-09', 'name': 'Franz-Peter Huhn-Gerlach', 'gender': 'M', 'city': 'Aachen', 'email': 'romualdeberth@hotmail.de', 'product_id': '48067560', 'amount_spent': 23.35}
{'transaction_date': '2021-02-14', 'name': 'Nurten Faust', 'gender': 'M', 'city': 'Hannover', 'email': 'claudia32@gmail.com', 'product_id': '95679921', 'amount_spent': 86.69}
{'transaction_date': '2021-02-06', 'name': 'Conny Finke', 'gender': 'F', 'city': 'Leipziger Land', 'email': 'fredoniemeier@gmx.de', 'product_id': '05454990', 'amount_spent': 20.36}
{'transaction_date': '2021-05-29', 'name': 'Prof. Linda Hertrampf', 'gender': 'M', 'city': 'Rehau', 'email': 'uortmann@patberg.de', 'product_id': '38261039', 'amount_spent': 90.4}
{'transaction_date': '2021-07-07', 'name': 'Pius Ehlert-Albers', 'gender': 'F', 'city': 'Leipziger Land', 'email': 'friedbert90@web.de', 'product_id': '22291349', 'amount_spent': 17.27}
{'transaction_date': '2021-03-10', 'name': 'Siglinde Rosemann', 'gender': 'M', 'city': '

In [9]:
dumps({'transaction_date': '2021-03-02', 'name': 'Katherina Mende', 'gender': 'M', 'city': 'Rochlitz', 'email': 'ludmila23@hotmail.de', 'product_id': '64859422', 'amount_spent': 91.66})

'{"transaction_date": "2021-03-02", "name": "Katherina Mende", "gender": "M", "city": "Rochlitz", "email": "ludmila23@hotmail.de", "product_id": "64859422", "amount_spent": 91.66}'

In [ ]:
# from time import sleep
# from json import dumps
# from kafka import KafkaProducer

# topic_name='test-data'
# producer = KafkaProducer(bootstrap_servers=['localhost:9092'],value_serializer=lambda x: dumps(x).encode('utf-8'))

# for e in range(1000):
#     data = {'number' : e}
#     producer.send(topic_name, value=data)
#     sleep(5)